In [1]:
import os
import glob

import DESutils
import GPRutils
import vK2KGPR
import plotGPR
import vK2KGPR
import vonkarmanFT as vk

import numpy as np
import astropy.units as u
import astropy.table as tb
from astropy.time import Time
import matplotlib.pyplot as plt
plt.style.use('~/GitHub/custom-matplotlib/custom.mplstyle')

from IPython import embed
from importlib import reload
import warnings
warnings.filterwarnings("ignore")

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
for expNum in zband:
    try:
        dataContainers = {
            "RCmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/dualOpt_NMmax/GPR.{expNum}.?RC.fits"),
            "RCLmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/L_BFGS_B_max/GPR.{expNum}.?RC.fits")
        }
    except Exception:
        continue

    print(f"{expNum}, {list(dataContainers.values())[0].band}")
    names = ["Method", "xi0", "xif", "Reduction", "Stars/arcmin^2", "K Variance", "Outer Scale", "Diameter", "Wind X", "Wind Y"]
    header = "".join([f"{name:<15}" for name in names])
    print(header)
    
    for i, (method, dC) in enumerate(dataContainers.items()):
        vals = [method]
        if i == 2:
            print()
        
        xi0 = dC.header["xi0"]
        Xerr = dC.header["xi0_Xerr"]
        Yerr = dC.header["xi0_Yerr"]
        xi0err = np.sqrt(Xerr**2 + Yerr**2)
        vals.append(f"{xi0:.2f} ± {xi0err:.2f}")

        xif = dC.header["xif"]
        Xerr = dC.header["xif_Xerr"]
        Yerr = dC.header["xif_Yerr"]
        xiferr = np.sqrt(Xerr**2 + Yerr**2)
        vals.append(f"{xif:.2f} ± {xiferr:.2f}")

        red = xi0/xif
        rederr = np.sqrt(((xi0err/xi0)**2 + (xiferr/xif)**2) * red**2)
        vals.append(f"{red:.2f} ± {rederr:.2f}")
        
        N = len(dC.TV[dC.TV["Maskf"]])
        starDensity = np.round((N / (3*u.deg**2)).to(u.arcmin**-2), 3)
        vals.append(starDensity.value)
        
        vals.extend(np.abs(np.around(dC.params, 7)))

        line = "".join([f"{param:<15}" for param in vals])
        print(line)
    print()
    
# a = b/c
# (da/a)^2 = (db/b)^2 + (dc/c)^2

348819, z
Method         xi0            xif            Reduction      Stars/arcmin^2 K Variance     Outer Scale    Diameter       Wind X         Wind Y         
RCmax          82.56 ± 1.76   5.11 ± 0.58    16.16 ± 1.87   0.926          157.5487455    0.998421       0.0204689      0.0096602      0.0338482      
RCLmax         82.56 ± 1.76   5.10 ± 0.58    16.18 ± 1.87   0.926          161.7125666    1.1631951      0.03           0.0060943      0.0243347      

355303, z
Method         xi0            xif            Reduction      Stars/arcmin^2 K Variance     Outer Scale    Diameter       Wind X         Wind Y         
RCmax          96.38 ± 2.38   5.46 ± 1.28    17.67 ± 4.16   0.983          658.7374416    0.0244126      0.4175583      0.1305385      0.2147242      
RCLmax         96.38 ± 2.38   4.92 ± 1.25    19.58 ± 4.99   0.983          495.0023374    0.1            0.4176481      0.0559151      0.0855853      

364209, z
Method         xi0            xif            Reduction      St